In [1]:
#coding=utf-8
'''
    FileName      ：bertDataProcess.ipynb
    Author        ：@zch0423
    Date          ：Jun 25, 2021
    Description   ：
'''
import jsonlines
import numpy as np
# import pickle
from tqdm import tqdm

In [2]:
path = "/Users/zch/Desktop/temp.nosync/output_tst.jsonl"

In [3]:
def getCLSW2V(sentence):
    '''
    @Description
    获取句子CLS对应的矩阵, 2*768
    ------------
    @Params
    sentence, dict, bert 返回单个句子
    ------------
    @Returns
    array, CLS value of two layers, [最后一层，倒数第二层]
    '''
    word = sentence["features"][0]
    if word["token"] != "[CLS]":
        raise ValueError("不是CLS")
    return np.array([word["layers"][0]["values"], word["layers"][1]["values"]])    

In [4]:
def getAvgW2V(sentence):
    '''
    @Description
    获取句子平均w2v, 2*768
    ------------
    @Params
    sentence, dict
    ------------
    @Returns
    array, [最后一层，倒数第二层]
    '''
    temp = []
    for word in sentence["features"]:
        temp.append(np.array([word["layers"][0]["values"], word["layers"][1]["values"]]))
    return np.mean(temp, axis=0)

In [5]:
def getW2V(path, w2v, outfile):
    '''
    @Description
    获取w2v
    ------------
    @Params
    path, str, jsonlines文件路径
    w2v, func, w2v处理方法
    outfile, numpy array pickle 保存路径
    '''
    data = []
    with jsonlines.open(path) as reader:
        for obj in tqdm(reader):
            data.append(w2v(obj))
    data = np.array(data)
    np.save(outfile, data)
    print(data.shape, "has been saved to ", outfile)

In [38]:
# with jsonlines.open(path) as reader:
#     for obj in reader:
#         print(len(obj["features"][0]["layers"][0]["values"]))
#         break

768


In [8]:
path_CLS = "/Users/zch/Desktop/IM319_NLP.nosync/pre_loaded/CLS_tst.npy"
getW2V(path, getCLSW2V, path_CLS)

4696it [00:25, 184.41it/s]


(4696, 2, 768) has been saved to  /Users/zch/Desktop/IM319_NLP.nosync/pre_loaded/CLS_tst.npy


In [9]:
path_avg = "/Users/zch/Desktop/IM319_NLP.nosync/pre_loaded/avg_tst.npy"
getW2V(path, getAvgW2V, path_avg)

4696it [00:32, 146.17it/s]


(4696, 2, 768) has been saved to  /Users/zch/Desktop/IM319_NLP.nosync/pre_loaded/avg_tst.npy


In [41]:
# test
data = np.load(path_CLS)
print(data.shape)

(35379, 2, 768)


In [42]:
data = np.load(path_avg)
print(data.shape)

(35379, 2, 768)


## 保留最后一层

In [6]:
import os
dir_path = "/Users/zch/Desktop/IM319_NLP.nosync/pre_loaded/"

In [7]:
def saveLast(infile, outfile):
    '''
    @Description
    保留bert最后一层, 导出到npy
    ------------
    @Params
    infile, str, path for npy
    outfile, str
    '''
    arr = np.load(infile)
    arr = np.delete(arr, 1, axis=1)
    arr.reshape(arr.shape[0], arr.shape[-1])
    np.save(outfile, arr)

In [10]:
with os.scandir(dir_path) as it:
    for entry in it:
        if not entry.name.endswith("npy"):
            continue
        saveLast(entry.path, "/Users/zch/Desktop/"+entry.name[:-4]+"_last.npy")

## test

In [23]:
infile = "/Users/zch/Desktop/IM319_NLP.nosync/project/data/w2v/avg.npy"
arr = np.load(infile)
arr = np.delete(arr, 1, axis=1)